# Initialize spark

In [1]:
## LOCAL ONLY

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.streaming import StreamingContext

spark = SparkSession \
    .builder \
    .appName("CloudProject") \
    .getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/17 19:29:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.types import *
scc = StreamingContext(spark,4)

# Initialize data streaming

In [3]:
MONITOR_SOURCE='s3://guglielmo-data-lake/monitor'
P_SENSORS_SOURCE='s3://guglielmo-data-lake/measurements/*/p*'
T_SENSORS_SOURCE='s3://guglielmo-data-lake/measurements/*/t*'

In [4]:
## LOCAL ONLY 

MONITOR_SOURCE='/home/guglielmo/universita/cloud_computing/progetto/code/mock-data/monitor'
P_SENSORS_SOURCE='/home/guglielmo/universita/cloud_computing/progetto/code/mock-data/measurements/pollution'
T_SENSORS_SOURCE='/home/guglielmo/universita/cloud_computing/progetto/code/mock-data/measurements/temperature'

### Initialize monitor data streaming

In [5]:
schema = StructType([
    StructField("device_id", StringType(), True),
    StructField("device_health", IntegerType(), True),
    StructField("type", StringType(), True),
    StructField("area", StringType(), True),
    StructField("customer", StringType(), True),
])
monitor_stream = spark.readStream.schema(schema).format("json").option('path',MONITOR_SOURCE).load()
monitor_stream

DataFrame[device_id: string, device_health: int, type: string, area: string, customer: string]

In [6]:
monitor_stream.isStreaming

True

### Initialize sensors data streaming

In [7]:
schema = StructType([
    StructField("device_id", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("humidity", IntegerType(), True),
    StructField("temperature", IntegerType(), True)
])
t_sensors_stream = spark.readStream.schema(schema).format("json").option('path',T_SENSORS_SOURCE).load()
t_sensors_stream

DataFrame[device_id: string, timestamp: timestamp, humidity: int, temperature: int]

In [8]:
schema = StructType([
    StructField("device_id", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("CO2_level", IntegerType(), True)
])
p_sensors_stream = spark.readStream.schema(schema).format("json").option('path',P_SENSORS_SOURCE).load()
p_sensors_stream

DataFrame[device_id: string, timestamp: timestamp, CO2_level: int]

In [9]:
print(monitor_stream.isStreaming, t_sensors_stream.isStreaming, p_sensors_stream.isStreaming)

True True True


In [10]:
monitor_query = monitor_stream \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

t_sensors_query = t_sensors_stream \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

p_sensors_query = p_sensors_stream \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

22/06/17 19:29:43 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-b74fe09d-1be9-473d-94ab-413230e3b4cb. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/06/17 19:29:43 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
22/06/17 19:29:44 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-9879d256-a723-4d30-9cc9-4e2f7ade8303. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/06/17 19:29:44 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not support

-------------------------------------------
Batch: 0
-------------------------------------------
-------------------------------------------
-------------------------------------------
Batch: 0
-------------------------------------------
Batch: 0
-------------------------------------------
+----------+--------------------+---------+
| device_id|           timestamp|CO2_level|
+----------+--------------------+---------+
|p-88074073|2022-06-17 17:27:...|      436|
+----------+--------------------+---------+

+----------+--------------------+--------+-----------+
| device_id|           timestamp|humidity|temperature|
+----------+--------------------+--------+-----------+
|t-67464570|2022-06-17 17:26:...|      40|         25|
+----------+--------------------+--------+-----------+

+----------+-------------+-----------+-----------+-------------+
| device_id|device_health|       type|       area|     customer|
+----------+-------------+-----------+-----------+-------------+
|t-17231251|     

In [ ]:
monitor_query.stop()
t_sensors_query.stop()
p_sensors_query.stop()

In [ ]:
extract_test = monitor_stream.select("device_health")
extract_test.writeStream\
  .outputMode("append")\
  .format("json")\
  .option("path", "s3://test-output-emr/")\
  .option("checkpointLocation", "s3://test-spark-checkpoints/")\
  .start().awaitTermination()

'''extract_test = t_sensors_stream.select("humidity")
extract_test.writeStream\
  .outputMode("append")\
  .format("json")\
  .option("path", "s3://test-output-emr/")\
  .option("checkpointLocation", "s3://test-spark-checkpoints/")\
  .start().awaitTermination()

extract_test = monitor_stream.select("CO2_level")
extract_test.writeStream\
  .outputMode("append")\
  .format("json")\
  .option("path", "s3://test-output-emr/")\
  .option("checkpointLocation", "s3://test-spark-checkpoints/")\
  .start().awaitTermination()'''